# Integrate an External Test Provider

This notebook demonstrates how to use a custom test provider to be able to use custom tests with the Validmind Developer Framework.
In the notebook, we load a couple different demo test providers and register them with the Validmind framework to be able to run a template that utilizes those tests.

In [1]:
# install validmind if its not already installed
# %pip install validmind

##### Update your template

Within the project you would like to use, create a new Content Block in the documentation template and add the following code:

```yaml
      - content_type: metric
        content_id: my_local_provider.tests.MyCustomTest
      - content_type: metric
        content_id: my_inline_provider.tests.MyCustomTest
```

See the section below on setting up and registering test providers for more info on how these content_id's get mapped to the actual test providers and tests.

### Setup the Validmind Connection

**get your api_key and api_secret from the Validmind UI**

In [4]:
import validmind as vm

vm.init(
    api_host = "http://localhost:3000/api/v1/tracking",
    api_key = "e22b89a6b9c2a27da47cb0a09febc001",
    api_secret = "a61be901b5596e3c528d94231e4a3c504ef0bb803d16815f8dfd6857fac03e57",
    project = "clj1vl7l9000lahztzgtj5aqj"
)

2023-06-20 15:08:36,290 - INFO - api_client - Connected to ValidMind. Project: External Test Provider Demo (clj1vl7l9000lahztzgtj5aqj)


### Preview the template for the current project to validate that it looks correct

In [5]:
# we should see two custom content blocks in the template whose IDs are under the namespaces registered below (`my_inline_provider` and `my_local_provider`)
# the ID should match the path to the `tests` directory in this repo
vm.preview_template()

Accordion(children=(VBox(children=(HTML(value='\n<div class="lm-Widget p-Widget jupyter-widget-Collapse jupyte…

### Register external test providers

We will now instantiate and register the Test Provider classes that we imported above.

For the Github provider, we will need to specify the `org` and `repo` to pull from as well as optionally pass a `token` if the repo is private.

For the Local Filesystem provider, we will just need to specify the root folder under which the provider class will look for tests. In this case, it is the current directory for this demo so you may have to adjust the path for your machine.

**Import the Local File System Test Provider from the `validmind.tests` module**

In [ ]:
from validmind.tests import LocalTestProvider

**Create an inline TestProvider Class that just returns a single test**

In [ ]:
class MySecondCustomTest(vm.vm_models.Metric):
    def description(self):
        return "This is a custom test from an external test provider."

    def run(self):
        return self.cache_results({"foo": "bar"})

    def summary(self, results):
        return vm.vm_models.ResultSummary(
            results=[
                vm.vm_models.ResultTable(
                    data=[{"results": results}],
                    metadata=vm.vm_models.ResultTableMetadata(
                        title="Custom Test from External Test Provider"
                    ),
                )
            ]
        )


class TestProviderInline:
    def load_test(self, test_id):
        # ignore the test_id and just return the single test above
        return MySecondCustomTest

In [6]:
# instantiate the test provider
inline_test_provider = TestProviderInline()
local_test_provider = LocalTestProvider(root_folder=".")

# register the test providers
vm.tests.register_test_provider(
    namespace="my_inline_provider",
    test_provider=inline_test_provider,
) # validmind will now call the `TestProviderInline.load_test` method whenever it encounters a test ID that starts with `my_inline_provider`
vm.tests.register_test_provider(
    namespace="my_local_provider",
    test_provider=local_test_provider,
) # validmind will now call the `LocalTestProvider.load_test` method whenever it encounters a test ID that starts with `my_local_provider`

### Running the template

Now we can run the template as usual and it will use the external test providers to load the appropriate tests.

In [13]:
vm.run_template()